# DIKERJAKAN DI KAGGLE
Runtime: 2 Jam-an

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import cv2
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from imgaug import augmenters as iaa

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data_dir = '/kaggle/input/files1/Malaria Cells'

In [ ]:
os.listdir(data_dir) 

In [ ]:
train_folder = data_dir+'/training_set'
test_folder = data_dir+'/testing_set'

# FEATURE EXTRACTION

In [ ]:
vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(130, 130, 3))
def extract_features(image):
    img = cv2.resize(image, (130, 130))
    img = np.expand_dims(img, axis=0)
    img = vgg16.predict(img)
    img = img.flatten()
    return img


In [ ]:
train_features = []
train_labels = []
test_features = []
test_labels = []

for label in ['Parasitized', 'Uninfected']:
    folder_path = os.path.join(train_folder, label)
    for filename in os.listdir(folder_path):
        if filename.endswith(".png"):
            image_path = os.path.join(folder_path, filename)
            image = cv2.imread(image_path)
            features = extract_features(image)
            train_features.append(features)
            train_labels.append(label)

In [ ]:
for label in ['Parasitized', 'Uninfected']:
    folder_path = os.path.join(test_folder, label)
    for filename in os.listdir(folder_path):
        if filename.endswith(".png"):
            image_path = os.path.join(folder_path, filename)
            image = cv2.imread(image_path)
            features = extract_features(image)
            test_features.append(features)
            test_labels.append(label)

In [ ]:
len(train_features)

# **MODELLING**

In [ ]:
k = 3
model = KNeighborsClassifier(n_neighbors=k)


In [ ]:
# Melakukan prediksi pada data validasi
model.fit(train_features, train_labels)




In [ ]:
test_predictions = model.predict(test_features)


In [ ]:
test_accuracy = accuracy_score(test_labels, test_predictions)
print("Test Accuracy:", test_accuracy)

In [ ]:
from sklearn.metrics import precision_score, recall_score

# Menghitung presisi
precision = precision_score(test_labels, test_predictions, pos_label='Parasitized')
print("Precision:", precision)

# Menghitung recall
recall = recall_score(test_labels, test_predictions, pos_label='Parasitized')
print("Recall:", recall)


In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [ ]:
all_features = train_features
all_labels = train_labels

kfold = KFold(n_splits=5, shuffle=True, random_state=42)

scores = cross_val_score(model, all_features, all_labels, cv=kfold)

print("Cross-Validation Accuracy:", scores.mean())